In [1]:
#!pip install selenium
#!pip install webdriver_manager

In [2]:
# this is the library that we will use to create break times in order to mimic human behaviour 
import time

# this library will be used for getting your credentials but keeping them hidden in the script 
from getpass import getpass

# Juicy stuff- these are the Classes we will use for interaction with a webpage:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException



# library for interacting with the operating system
import os

# you know pandas it's your best buddy 
import pandas as pd

# library for directory location:

import pathlib
import os
from os.path import join

#Ignore warning -- Some methods are going to be deprecated and I didn't change all (mainly in the function scrapper)
import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:02<00:00, 3.16MB/s]


In [3]:
#open the website

driver.get('https://www.linkedin.com')

In [4]:
# delete the "Accept cookies pop-up notification":

#signin = driver.find_element(By.XPATH,'//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]')
#signin.click()

In [5]:
# finding the sign in element

#signin = driver.find_element_by_class_name('nav__button-secondary')
#or new way 
signin = driver.find_element(By.CLASS_NAME, 'nav__button-secondary')

signin.click() ## with the method click you will be able to click the element you searched for ##

time.sleep(1.5) ## never forget to put a sleep timer in order to mimic human interaction ##

In [6]:
# login into your LinkedIn account

# input your email

email = input('Enter your email : ')

# here we are going to find the box where we input the email
email_box = driver.find_element(By.ID, "username") #email_box = driver.find_element_by_id("username") #deprecated

#clear the search box if it has already something in there
email_box.clear()

# with the method send_keys() you can send the information from the script to the webpage
email_box.send_keys(email)
time.sleep(2)

# input the password
password = getpass('Enter your password : ')

# here we are going to find the box where we input the password
pass_box = driver.find_element(By.ID, 'password') #pass_box = driver.find_element_by_id('password') #deprecated

#clear the paword box if it has already something in there
pass_box.clear()

# here we will send the password to the driver 
pass_box.send_keys(password)
time.sleep(1)

login = driver.find_element(By.CLASS_NAME,'login__form_action_container') #login = driver.find_element_by_class_name('login__form_action_container') #deprecated

login.click()
time.sleep(1)

Enter your email : quino117@hotmail.com
Enter your password : ········


In [7]:
#Disable messages tab:

remove_message = driver.find_element(By.XPATH, '/html/body/div[6]/aside/div[1]/header/div[3]/button[2]')
remove_message.click()

In [8]:
# search bar

# locate the element by the id

# or new method
job_icon = driver.find_element(By.CSS_SELECTOR, "span[title='Jobs']")

job_icon.click()
time.sleep(2)



#global-nav > div > nav > ul > li:nth-child(3) > a

In [9]:
# this is not the best approach because even with selenium some elements keep on changing (automatically update themselves)  
# because they are written in
# ember.js 

search_job = driver.find_elements(By.CLASS_NAME,'jobs-search-box__text-input')[0] #search_job = driver.find_elements_by_class_name('jobs-search-box__text-input')[0]

job = input('What job do you want to search for: ')

search_job.send_keys(job)
time.sleep(2)

#search_job.send_keys(Keys.TAB)


What job do you want to search for: data


In [10]:
#Use the Tab key from the keyboard because it allows us to go from the job box to the location box

search_job.send_keys(Keys.TAB)
#get the element the cursor is on
location_box = driver.switch_to.active_element

location = input('Where do you want to search for jobs: ')
location_box.send_keys(location)

time.sleep(1)


Where do you want to search for jobs: Spain


In [11]:
# click enter with the parameter keys.ENTER inside the method send_keys()

location_box.send_keys(Keys.ENTER)

In [12]:
# Maximize the window 

driver.maximize_window() 

## you can also fullscreen the window 

#driver.fullscreen_window()

In [13]:
# go to the end of the page in order for all the elements to be loaded
## to scroll a section you should find an a tag in that section, click that element and then scroll to the end

page = driver.find_element(By.CSS_SELECTOR, "div[class^='application-outlet']") 
page.click()
time.sleep(1)

job_viewer = driver.find_element(By.CSS_SELECTOR, "a[class^='disabled ember-view']") #page = driver.find_element_by_css_selector("a[class^='disabled ember-view']")
job_viewer.send_keys(Keys.END)

In [14]:
## build a function ##

# GET ALL JOB LINKS 

''' !!!ADVANCED: talk a little about how to find with partial class name!!!

e.g.:
      driver.find_elements_by_css_selector(<tagname>"[<attribute>^='<partial text of the attribute>']")

exammple in the line bellow '''

job_raw = driver.find_elements(By.CSS_SELECTOR,"a[class^='disabled ember-view']")

def scrapper(numb_j = len(job_raw)): ## add pages

    """ SUMMARY: This function retrives all the job posts 
    links from one page and returns a dataset with
    the name of the job in one column and the link to the post in the other. 
    Also it will write the same info in different files
    for every single job post.
    
    HOW IT WORKS: Input the number of jobs you want to scrape. 
    It will search on the page for the elements by css selector 
    from all the job posts then loop for every single element and retrieve the 'href'. 
    Also it will click on every job post and find the job name.
    This info will be saved in a dictionary that will in the end be converted to 
    a dataset.
    Below we will open and create a text file with the name of the job post 
    and inside save the link for further details"""

    # For scraper reasons it's required to duplicate the numb_j as it retrieves 2 times the same position:
    numb_j = numb_j*2
    
    # empty list for saving the job names , link and extra info: 
    job_list = []

    # reduce the page size in order to be able to find the name of the job in the right session 
    # driver.execute_script("document.body.style.zoom='67%'")

    # all jobs in the page
    job_raw = driver.find_elements(By.CSS_SELECTOR,"a[class^='disabled ember-view']") 

    # go to the end of the page for all the elements to be loaded
    page = driver.find_element(By.CSS_SELECTOR,"a[class^='disabled ember-view']")
    page.send_keys(Keys.END)   
    # go to the top of the page for all the elements to be loaded
    page.send_keys(Keys.CONTROL + Keys.HOME) # combination of the two keys brings you to the top of the element


    for j_idx in range(min(numb_j, len(job_raw))):
        if j_idx % 2 == 0:
            # get the job link
            ref = job_raw[j_idx].get_attribute('href') 
            time.sleep(2)

            # increase the page size because the inspect for getting the job name where done wiht the page maximized
            driver.execute_script("document.body.style.zoom='100%'")

            ## let's click on the job post ##
            # driver.find_elements_by_css_selector("a[class^='disabled ember-view']")[j_idx].click()
            job_raw[j_idx].click()
            time.sleep(1.4)

            ## then we reduce the page size in order to be able to see the right part of the page 
            # and find the element with the name of the job ##
            driver.execute_script("document.body.style.zoom='67%'")
            time.sleep(2.3)

            # get the job name with the .text method
            job_name = driver.find_element(By.CSS_SELECTOR,"h2[class^='t-24 t-bold']").text
            time.sleep(2.2)

            # get job description with css selector + .text method:

            # get company name:
            company_name = driver.find_element(By.CSS_SELECTOR,"span[class^='jobs-unified-top-card__company-name']").text
            
            # get job details: 

            job_details = driver.find_element(By.ID,"job-details").text
            
            # get salary if available: 

            try:
                skills_missing = driver.find_element(By.XPATH,"/html/body/div[6]/div[3]/div[4]/div/div/main/div/div[2]/div/div[2]/div[1]/div/div[4]/section[2]/div/div/div[2]/div/a").text
            except NoSuchElementException:
                skills_missing = None
            
            # get workplace details: 

            try:
                workplace_type = driver.find_element(By.CSS_SELECTOR,"span[class^='jobs-unified-top-card__workplace-type']").text
            except NoSuchElementException:
                workplace_type = None
            
            # get skills: 

            try:
                skills = driver.find_element(By.CSS_SELECTOR,".app-aware-link.job-details-how-you-match__skills-item-subtitle.t-14.overflow-hidden").text
            except NoSuchElementException:
                skills = None
            
            # get insights: 

            insights = driver.find_element(By.CSS_SELECTOR,"li[class^='jobs-unified-top-card__job-insight']").text

            # increase the page size: 
            driver.execute_script("document.body.style.zoom='100%'")

            # populate list:
            job_idx_list = [ref, job_name , company_name , job_details, skills_missing, workplace_type, skills, insights]
            time.sleep(3)

            # page.send_keys(Keys.PAGE_DOWN)

            job_list.append(job_idx_list)

            print(f"Collected job : {job_name} for company: {company_name}")

    #Create dataframe:
    Job_dataframe = pd.DataFrame(job_list, 
                                 columns = ["job_link", "position", "company name", "job description","skills_missing", "workplace_type", "skills", "insights"]
                                ).drop_duplicates()


    #Save dataframe in excel file to later use our job 
    Job_dataframe.to_excel(pathlib.Path().joinpath('scraped_jobs.xlsx'), 
                           sheet_name='Jobs',
                           index= False)
    
    return Job_dataframe

In [15]:
# How many job posts do you want to search for:
## let's keep it a low number because it takes some time and I don't get to have my account blocked :D ## 
jobsN = int(input('How many job posts you want to retrieve: '))

d1 = scrapper(jobsN)

How many job posts you want to retrieve: 15
Collected job : Internet of Things Researcher for company: Vicomtech
Collected job : Analista Business Intelligence for company: nettrim technology
Collected job : Data Analyst for company: Vermut
Collected job : Junior AI Researcher for Industry (Bilbao) for company: Vicomtech
Collected job : ANALISTA DE DATOS (H/M/X) for company: Manpower


In [16]:
d2 = scrapper(10)

Collected job : Analista de datos for company: 
Collected job : Desarrollador/a BI for company: FI Group
Collected job : Business Intelligence Specialist for company: EDP Renewables
Collected job : Data Analyst for company: join.com
Collected job : Data Scientist for company: Merkle Spain
Collected job : Data Engineer for company: El Corte Inglés
Collected job : Credit Data Analyst with English and Power BI - PageGroup SSC for company: Page Personnel
Collected job : Local Data Analyst for company: Eli Lilly and Company
Collected job : Analista de datos for company: ALPHABET CONSULTING
Collected job : Power BI Specialist for company: ERNI


In [17]:
d3 = scrapper(15)

Collected job : BI Engineering Analyst for company: The Workshop - Inventors of Play
Collected job : Product Analyst for company: Tripledot Studios
Collected job : Junior Data Scientist for company: Deloitte
Collected job : Data Warehouse Moduler, Remote Spain for company: Hotelbeds
Collected job : JUNIOR PROPERTY ANALYST for company: SPRINTER
Collected job : Business Intelligence - Q2 for company: VISEO IBERIA
Collected job : Online Data Analyst for company: TELUS International
Collected job : Associate - Data Research for company: Clarity AI
Collected job : Product Data Analyst for company: SumUp
Collected job : Online Data Analyst for company: TELUS International
Collected job : Junior Logistics Analyst for company: Amazon
Collected job : Desarrollador Junior Power BI for company: EXCELIA
Collected job : Business Digitalization Analyst (Contrato en prácticas) for company: ENGIE España


In [18]:
d4 = scrapper(20)

Collected job : Data Scientist for company: agap2 Spain
Collected job : Data Consultant for company: SDG Group España
Collected job : Online Data Analyst - Spain for company: TELUS International
Collected job : Data Analyst / Data Engineer for company: Inetum
Collected job : Master Data Specialist for company: Wodker Associates
Collected job : Operations Data Analyst (They/She/He) for company: Glovo
Collected job : Finance Data Analyst for company: Guardian Industries
Collected job : Digital Analyst for company: NTT DATA Europe & Latam
Collected job : Financial Planning and Analysis (FP&A) Analyst for company: Lookiero
Collected job : Junior Digital Analyst Freelancer for company: Roche
Collected job : Corporate Data Analyst for company: Leroy Merlin
Collected job : Finance Data Analyst for company: Guardian Industries
Collected job : Associate - Data Research for company: Clarity AI


In [19]:
d5 = scrapper(15)

Collected job : Técnico(a) de Sostenibilidad con Data Analytics for company: VidaCaixa
Collected job : Power BI Expert Developer for company: NTT DATA Europe & Latam
Collected job : Data Analyst of AML Transaction Monitoring Quality Control for company: Santander
Collected job : Digital Analyst for company: NTT DATA Europe & Latam
Collected job : Route Optimization Analyst for company: CHEP
Collected job : Junior Data Scientist - Hybrid Intelligence for company: Capgemini Engineering
Collected job : Sales, Trading, Structuring Analyst - Expert Graduate Programme - 2023 for company: Barclays Corporate & Investment Bank
Collected job : CONSULTOR BI – DATA ENGINEER for company: VISEO IBERIA
Collected job : Técnico/a de migraciones - Data Analyst for company: ODILO
Collected job : Online Data Analyst - Spain for company: TELUS International AI Data Solutions
Collected job : Control de Gestión junior for company: Confidencial
Collected job : Junior Operations Engineer for company: Atlantica

In [20]:
d6 = scrapper(15)

Collected job : Desarrollador Junior Business Central for company: Integra
Collected job : Total Rewards Junior Analyst for company: Preply
Collected job : Workforce Management Analyst for company: Teleperformance
Collected job : Digital Analytics Specialist for company: Schindler Group
Collected job : Data Engineer (remote) for company: Guardian Industries
Collected job : Data Specialist Spark / Scala - Full Remote for company: Profile Software Services
Collected job : Data Engineer (remote) for company: Guardian Industries
Collected job : Data Engineer _ Analytics and Modeling analyst for company: Accenture España
Collected job : Deal Desk Analyst for company: Infor
Collected job : CRO /Digital analyst for company: Habitissimo
Collected job : Master Data Analyst for company: SGS
Collected job : Business Analyst | Reporting Improvements SAS Visual Analytics for company: CORUS Consulting
Collected job : Digital &Technology Product Manager Internship for company: CBRE España


In [21]:
d7 = scrapper(15)

Collected job : Business Development Analyst in Madrid. for company: Deoleo
Collected job : Data Scientist - Hybrid Intelligence for company: Capgemini Engineering
Collected job : Onboarding Product Associate for company: Western Union
Collected job : Data Analyst Sector Energético (Madrid/Barcelona) - ID 8001 for company: Naturgy
Collected job : Data Analyst - Courier Growth (They/She/He) for company: Glovo
Collected job : Data Engineer (remote) for company: Guardian Industries
Collected job : Data Strategy Manager for company: Clarity AI
Collected job : Project Assistant Internship for company: Arcadis
Collected job : Data Scientist - HEALTH for company: Capgemini Engineering
Collected job : Finance Business Analyst for company: Stryker
Collected job : Jr Account & Ad Ops for company: Seedtag
Collected job : Data Scientist - Valencia - for company: Michael Page
Collected job : Data Engineer for company: Cívica


In [22]:
d8 = scrapper(15)

Collected job : ANALISTA BUSINESS INTELLIGENCE for company: LA SIRENA
Collected job : Analista Business Intelligence for company: nettrim technology
Collected job : Business Analyst (They/She/He) for company: Glovo
Collected job : Data Analyst for company: Quantion
Collected job : Business Analyst for company: Nucleoo
Collected job : Business Analyst (Barcelona) for company: TIA JOBS
Collected job : Business Analyst for company: Deporvillage.com
Collected job : Business Analyst for company: NBCC Consulting
Collected job : Business Analyst for company: HP
Collected job : Junior Data Analyst Power BI - CONTRATO 6 MESES for company: LogiRAIL
Collected job : Business Intelligence Consultant for company: ÁlamoConsulting
Collected job : Business Analyst for company: Kiwi.com
Collected job : Data Analyst for company: SHA Wellness Clinic


In [23]:
d9 = scrapper(15)

Collected job : Analista de datos for company: Master Work, S.A.
Collected job : Consultor BI for company: Capgemini Engineering
Collected job : Data Business Support Analyst - SCF Global Services for company: Santander
Collected job : Data Analyst for company: Grupo Caja de Ingenieros
Collected job : Data Analyst for company: SDG Group España
Collected job : Business Analyst for company: HCLTech
Collected job : Business Analyst - Empresa Industrial for company: Selection by Eurofirms
Collected job : Data Analyst Power BI for company: CGI
Collected job : Business Analysis H/M for company: Inetum
Collected job : Data Analyst (80% Teletrabajo) for company: Oxigent Technologies
Collected job : Business Analyst SALESFORCE for company: Capgemini
Collected job : Analista de negocio for company: Securitas Direct by Verisure
Collected job : Data Analyst for company: HCLTech


In [24]:
d10 = scrapper(15)

Collected job : AWS Business Intelligence Analyst for company: TD SYNNEX
Collected job : Business Analyst Laboratory Informatics for company: Synthon
Collected job : Business Analyst con Ingles (H/M) for company: Michael Page
Collected job : Customer Intelligence Analyst for company: Zurich Insurance
Collected job : CRT Data Management and Business Analyst Specialist for company: HP
Collected job : Business Analyst Hybrid & Energy Storage Systems (d|m|f) for company: BayWa r.e. EMEA
Collected job : Data Analyst - DataLake (Openbank) for company: Openbank
Collected job : Product Data Analyst for company: SumUp
Collected job : UTS Business Analyst for company: Unisys
Collected job : Finance Business Analyst for company: Stryker
Collected job : Data Analyst for company: Octopus Energy
Collected job : Business Analyst in Securities Services-M/F/D for company: Societe Generale Securities Services - SGSS
Collected job : Data Science Analyst for company: Locaria


In [25]:
d11 = scrapper(15)

Collected job : Desarrollador de inteligencia empresarial for company: Obliku
Collected job : BI Analyst (Pricing) for company: Makro España
Collected job : Business Analyst DAM (Digital Asset Management) & Analytics - Product Information & Content for company: Nestlé
Collected job : Senior Game Data Analyst for company: Tilting Point
Collected job : Operations Data Analyst (They/She/He) for company: Glovo
Collected job : Business Analyst for company: Keywords Studios
Collected job : Multilingual Graduate program – Data analysis for company: Morningstar
Collected job : Calypso Business Analyst for company: First Derivative
Collected job : 2023 - BUSINESS ANALYST for company: Kearney
Collected job : Data Analyst - Chatbot for company: ING España&Portugal
Collected job : Data Analyst (Engineer) for company: Revolut
Collected job : Business Analyst - Quotation CPQ for company: Schneider Electric
Collected job : Senior Data Analyst for company: Tinámica


In [26]:
d12 = scrapper(15)

Collected job : BI Developer for company: Saber.tech
Collected job : Business Analyst SAP for company: Scalian
Collected job : Country Compliance Business Analyst for company: Openbravo
Collected job : BI developer (con Azure) for company: Plain Concepts
Collected job : Data Analyst Sector Energético (Madrid/Barcelona) - ID 8001 for company: Naturgy
Collected job : Senior Bi Analyst for company: SCRM - Lidl International Hub
Collected job : Senior Data Analyst for company: TravelPerk
Collected job : Language Data Analyst - French for company: TransPerfect
Collected job : Business Analysis and Sales Planning (idioma francés) for company: Page Personnel
Collected job : Multilingual Associate Data Research Analyst for company: Morningstar
Collected job : Biostatistician II for company: Precision For Medicine
Collected job : Business Analyst for company: Capgemini
Collected job : REMOTE WORK for Spanish Speakers | Data Analyst for company: TELUS International AI Data Solutions


In [27]:
d13 = scrapper(15)

Collected job : Business Intelligence Developer for company: Futurs
Collected job : Power BI Expert Developer for company: NTT DATA Europe & Latam
Collected job : BI Analyst for company: LLYC
Collected job : Experto/a Power BI, ETLs y SQL (Inglés alto) for company: Inetum
Collected job : Business Intelligence (Tableau) Developer ESP for company: Teladoc Health
Collected job : Web Business Analyst for company: Omega CRM, A Merkle Company
Collected job : Danish Language Data Analyst for company: TELUS International
Collected job : Customer Insights Analyst (Bangkok Based, Relocation Provided) for company: Agoda
Collected job : Data Analyst & BI developer for company: Siemens Gamesa
Collected job : Senior Biostatistician (CDS) EMEA for company: Labcorp
Collected job : Senior Biostatistician (CDS) EMEA for company: Labcorp
Collected job : Business Analyst (Bangkok Based, Relocation Provided) for company: Agoda
Collected job : Biostatistician II for company: Precision Medicine Group


In [28]:
d14 = scrapper(15)

Collected job : WEB ANALYTICS SPECIALIST for company: Reverse
Collected job : Testing Analyst - Sales Practice Monitoring for company: American Express
Collected job : Senior Cost Performance Analyst for company: Hotelbeds
Collected job : Business Intelligence Data Engineer for company: SDG Group España
Collected job : ANALISTA DE PLANES PROMOCIONALES for company: Grupo IskayPet (Tiendanimal, Kiwoko, Kivet, Clinicanimal)
Collected job : Business Intelligence & Operations for company: Adobe
Collected job : Ingeniero de datos for company: Arte Consultores
Collected job : Junior Credit Analyst for company: Elavon Europe
Collected job : ANALISTAS PROGRAMADORES NATURAL ADABAS for company: Inetum
Collected job : Online Data Analyst - Spain for company: TELUS International
Collected job : Data Analyst - Sector Financiero for company: Octopus Energy
Collected job : Analistas Funcionales COBOL for company: Inetum
Collected job : Financial Analyst - Part time for company: Moodle


In [29]:
d15 = scrapper(15)

Collected job : Data and Technical Operations Engineer for company: Clarity AI
Collected job : Senior - Cyber Data Science for company: Deloitte
Collected job : Digital Analyst eCommerce for company: Beam Suntory
Collected job : Junior Portfolio Analyst for company: Cabot Financial Spain
Collected job : Senior Business Intelligence Analytics for company: Cofidis España
Collected job : Antipiracy Analyst (Korean/Japanese speakers) for company: Red Points
Collected job : Analista financiero para entidad bancaria for company: Fundación Universia
Collected job : SPAIN - ONLINE DATA ANALYST for company: TELUS International AI Data Solutions
Collected job : .Net Developer - AI & Video Analytics Company Vallés Occ. for company: Page Personnel


In [30]:
d16 = scrapper(15)

Collected job : Data Engineer for company: Apex Group Ltd
Collected job : Analista compra y coberturas mercados de energía for company: Creara, Energy Experts
Collected job : Shopper Insights Analyst for company: Coca-Cola Europacific Partners
Collected job : BIG DATA ENGINEER - Hybrid Intelligence for company: Capgemini Engineering
Collected job : BA - Market Distribution for company: Scalian
Collected job : Data Engineer for company: Atradius
Collected job : IT PMO for company: Sopra HR Software
Collected job : Junior Associate - McKinsey Digital for company: McKinsey & Company
Collected job : Big Data Engineer – Data Lake – Openbank (Openhub Bilbao) for company: Santander
Collected job : Business Analyst (Bangkok Based, Relocation Provided) for company: Agoda
Collected job : Soporte de Análisis de Crédito for company: Sofinco España
Collected job : NPS Voice of Customer Analyst for company: BNP Paribas Personal Finance
Collected job : Regulatory Data Analyst for company: BME | Bolsa

In [31]:
d17 = scrapper(15)

Collected job : SAP SD Business Analyst for company: ALTEN
Collected job : ETL Developer for company: TransPerfect
Collected job : Data Engineer (Machine Learning) for company: GMV
Collected job : Application Support Analyst for company: Western Union
Collected job : Consultor/a Junior Financial Analytics for company: KPMG España
Collected job : Data Analyst for company: ToBeIT
Collected job : Analista de Negocio de Contact Center for company: Evolutio Empowering the cloud
Collected job : Global IT Master Data (SAP MDG-F) Business Solution Analyst for company: Roche
Collected job : ETL Developer (PySpark) for company: atSistemas
Collected job : Data Center Critical Environment Technician. for company: Microsoft
Collected job : Power BI y SQL - Recién Graduado/a en Ingeniería Informática, Telecomunicaciones y/o Telemática for company: Minsait
Collected job : Data System Analyst for company: ContourGlobal
Collected job : Financial Analyst for company: CHEP


In [32]:
d18 = scrapper(15)

Collected job : Security Architect for company: The TMRW Foundation
Collected job : Engineering Test&Validation Design Leader for company: Valeo
Collected job : Data Analyst for company: Vermut
Collected job : UTS Business Analyst for company: Unisys
Collected job : Sales Analyst for company: Brown-Forman
Collected job : Business Analyst (Barcelona) for company: TIA JOBS
Collected job : Data Analyst for company: Product Madness 📱🎮
Collected job : Senior Product Analyst for company: Scopely
Collected job : Business Analyst - Future Opportunities for company: Unisys
Collected job : Web Analytics Manager for company: Merkle EMEA
Collected job : Big Data Arch for company: Accenture España
Collected job : Mid Data Analyst/Engineer at Factorial HR for company: join.com
Collected job : Principal Biostatistician, home-based EMEA for company: Labcorp


In [33]:
fourth = pd.concat([d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18], ignore_index=True)

In [34]:
fourth

,job_link,position,company name,job description,skills_missing,workplace_type,skills,insights
0,https://www.linkedin.com/jobs/view/3527504347/...,Internet of Things Researcher,Vicomtech,About the job\nVicomtech (www.vicomtech.org) i...,"Kubernetes, Docker, Arquitectura de sistemas, ...",On-site,"Python, Inglés, Comunicación, and Investigación",Full-time · Mid-Senior level
1,https://www.linkedin.com/jobs/view/3570080991/...,Analista Business Intelligence,nettrim technology,About the job\nEn nettrim technology buscamos ...,"Almacenamiento de datos, Expresiones de anális...",Hybrid,"SQL, Inteligencia empresarial, and Trabajo en ...",Full-time · Associate
2,https://www.linkedin.com/jobs/view/3591654262/...,Data Analyst,Vermut,About the job\nAbout Vermut\n\nVermut is a com...,None,On-site,"Análisis de datos, Analítica de datos, Capacid...",Full-time
3,https://www.linkedin.com/jobs/view/3510600361/...,Junior AI Researcher for Industry (Bilbao),Vicomtech,About the job\nVicomtech (www.vicomtech.org) i...,"Aprendizaje automático, Kubernetes, Docker, Ci...",On-site,"Inglés, Python, Comunicación, Análisis de dato...",Full-time · Mid-Senior level
4,https://www.linkedin.com/jobs/view/3590620667/...,ANALISTA DE DATOS (H/M/X),Manpower,About the job\nDesde Manpower buscamos Logisti...,"Pedidos de compra, Facturación, and Intermedia...",On-site,"Microsoft Office, Microsoft Excel, Resolución ...",Full-time
...,...,...,...,...,...,...,...,...
214,https://www.linkedin.com/jobs/view/3524208459/...,Business Analyst - Future Opportunities,Unisys,About the job\nWe Believe in Better!\n\nWe are...,None,None,"Necesidades empresariales, Análisis de negocio...",Full-time · Mid-Senior level
215,https://www.linkedin.com/jobs/view/3532606140/...,Web Analytics Manager,Merkle EMEA,About the job\nCompany Description:\n\nIntrodu...,"Google Tag Manager, Análisis web, Reflexión es...",On-site,"Atención al detalle, Comunicación, and Analítica",Full-time · Mid-Senior level
216,https://www.linkedin.com/jobs/view/3462714653/...,Big Data Arch,Accenture España,About the job\nDescription\n\nEsta es tu oport...,"Big data, Ingeniería de datos , and Modelado d...",Hybrid,"Analítica, Analítica de datos, Bases de datos,...",Full-time · Mid-Senior level
217,https://www.linkedin.com/jobs/view/3593355780/...,Mid Data Analyst/Engineer at Factorial HR,join.com,About the job\nHi there 👋\n\nAs Factorial cont...,"Resolución creativa de problemas, Hormigón, Am...",Remote,Comunicación and Analítica,Full-time · Entry level


In [35]:
fourth.to_csv('fourth.csv', index=False)